In [ ]:
################################
########### IMPORTS ############
################################

In [ ]:
from pycbc import distributions
from pycbc.waveform import get_td_waveform, td_approximants
from pycbc.detector import Detector
import matplotlib.pyplot as plt
import numpy as np
import gwpy
import pylab
from tqdm.notebook import tqdm
from gwpy.timeseries import TimeSeries
import pandas as pd
import os
import csv
import pycbc.noise
import pycbc.psd
from pycbc.filter import matched_filter
import random

In [ ]:
##########################################
########### CONFIG PARAMETERS ############
##########################################

In [ ]:
DATA_PATH = "/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/models/debugging/Missing Data Imputation/Test 1/data/"
NO_OF_TRAINING_SAMPLES = 1000
NOISE_DURATION = 10             # in min
TOTAL_SIGNAL_DURATION = 8       # in min
EACH_SIGNAL_DURATION = 5        # in sec
SIGNAL_SPACING = 2              # in sec
MISSING_DURATION = 0.2          # in sec
COSINE_FREQ_LBOUND = 100
COSINE_FREQ_UBOUND = 4096
SIGNAL_AMPLITUDE_SUPPRESSION = 0.2 # 0.2

In [ ]:
##################################################
########### COLOURED NOISE GENERATION ############
##################################################

In [ ]:
def generateColoredNoise():
    # The color of the noise matches a PSD which you provide
    flow = 30.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    # Generate 4 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(NOISE_DURATION*60 / delta_t)
    noise = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)

    noise *= 1e21
    noise = TimeSeries.from_pycbc(noise)

    return noise

In [ ]:
# noise = generateColoredNoise()
# noise.plot()

In [ ]:
###############################################
########### COSINE WAVE GENERATION ############
###############################################

In [ ]:
# Trash Code

# time = TimeSeries(np.arange(0, 100), sample_rate=4096)
# time = time.to_value()
# wave = np.cos(time)
# plt.plot(time, wave)
# plt.show()

In [ ]:
# This function generates cosine wave 
# of given duration, frequency with 
# randomly sampled amplitude and start time
def generateRandomCosine(duration, frequency):
    x_right_bound = int(60*(1/frequency)*4096) + 1 # sets the right hand bound for x values such that 60 sec of signal data is generated
    x = np.arange(0, x_right_bound, (1/frequency))
    
    random_amp_scale = np.random.random() if np.random.random() else 1
    
    wave = SIGNAL_AMPLITUDE_SUPPRESSION*random_amp_scale*np.cos(x)
    wave = TimeSeries(wave, sample_rate=4096).to_pycbc()

    random_start = int(np.random.random()*(60 - duration))
    
    wave = TimeSeries.from_pycbc(wave.time_slice(random_start, random_start+duration))
    wave.t0 = 0

    return wave

In [ ]:
# cosine = generateRandomCosine(5, 100)
# cosine.plot()

In [ ]:
# This function generates array of
# cosine wave using generateRandomCosine
# with random frequency
def generateSignals():
    signals = []
    n_signals = int(((NOISE_DURATION - TOTAL_SIGNAL_DURATION)*60)/SIGNAL_SPACING) + 1

    for i in range(n_signals):
        r = np.random.random() if np.random.random() else 1
        random_freq = int(r*COSINE_FREQ_UBOUND) + COSINE_FREQ_LBOUND
        signals.append(generateRandomCosine(EACH_SIGNAL_DURATION, random_freq))

    return signals

In [ ]:
# s = generateSignals()
# s[27].plot()

In [ ]:
###############################################
########### SAMPLE DATA GENERATION ############
###############################################

In [ ]:
# This function generates
# one data sample with 
# every signal injected at
# random start time in noise

# def generateDataSample():
#     data = generateColoredNoise()
#     signals = generateSignals()

#     for signal in signals:
#         random_start = int(np.random.random()*(TOTAL_SIGNAL_DURATION*60 - EACH_SIGNAL_DURATION - 2))

#         signal.t0 = random_start
#         data = data.inject(signal)

#     return data

In [ ]:
# data = generateDataSample()
# data.plot()

In [ ]:
# This function generates
# one data sample with 
# every signal injected with
# equal seperation till the end
def generateDataSample():
    data = generateColoredNoise()
    signals = generateSignals()
    
    start = 0
    seperation = EACH_SIGNAL_DURATION + SIGNAL_SPACING

    for signal in signals:
        signal.t0 = start
        start += seperation

        data = data.inject(signal)

    return data

In [ ]:
data = generateDataSample()
data.plot()

In [ ]:
#####################################################
########### MISSING DATA MASK GENERATION ############
#####################################################

In [ ]:
# print(random.sample([0, 1], counts=[MISSING_DURATION*60*4096, (NOISE_DURATION - MISSING_DURATION)*60*4096], k=NOISE_DURATION*60*4096))

In [ ]:
data = generateDataSample()
data1 = data.copy()
l = [1 for i in range(len(data1.value))]
temp = data1.value
for i in range(0, 4096*60):
    l[i] = 0

temp *= l
# new_data = TimeSeries(new_data, sample_rate=4096)
# new_data.plot()
data1.plot()
data.plot()

In [ ]:
##############################################
########### FINAL DATA GENERATION ############
##############################################

In [7]:
f = open(DATA_PATH + "data.csv", 'w')
g = open(DATA_PATH + "mask.csv", 'w')
cw1 = csv.writer(f)
cw2 = csv.writer(g)

data = generateDataSample()
cw1.writerow(data.value)
f.close()

cw2.writerow([MISSING_DURATION])
for i in tqdm(range(NO_OF_TRAINING_SAMPLES)):
    random_mask_start = int(np.random.random()*4096*(NOISE_DURATION*60 - MISSING_DURATION - 1))
    cw2.writerow([random_mask_start])

g.close()

  0%|          | 0/1000 [00:00<?, ?it/s]